In [8]:
import os
from pathlib import Path
from shutil import copyfile
from datetime import datetime
import pygeohydro as gh
import pygeoutils as geoutils
from pynhd import NLDI
from utils.read_files import read_from_control, make_default_path
import geopandas as gpd
import numpy as np
import xarray as xr
import rioxarray as rxr
import scipy.stats as sc

In [2]:
# Store the name of the 'active' file in a variable
controlFile = 'control_EastRiver.txt'

In [3]:
# Forcing shapefile path & name
land_path = read_from_control(controlFile,'parameter_land_mode_path')
land_name = read_from_control(controlFile,'parameter_land_tif_name')

In [4]:
# Specify default path if needed
if land_path == 'default':
    land_path = make_default_path('parameters/landclass/', controlFile) # outputs a Path()
else:
    land_path = Path(land_path) # make sure a user-specified path is a Path()

In [7]:
nlcd_to_modis = {
    11: 14,  # Open Water → Open Water
    12: 13,  # Perennial Ice/Snow → Barren
    21: 12,  # Developed, Open Space → Urban and Built-Up
    22: 12,  # Developed, Low Intensity → Urban and Built-Up
    23: 12,  # Developed, Medium Intensity → Urban and Built-Up
    24: 12,  # Developed, High Intensity → Urban and Built-Up
    31: 13,  # Barren Land (Rock/Sand/Clay) → Barren
    41: 5,   # Deciduous Forest → Mixed Forest
    42: 1,   # Evergreen Forest → Evergreen Needleleaf Forest
    43: 5,   # Mixed Forest → Mixed Forest
    52: 6,   # Shrub/Scrub → Shrubland
    71: 7,   # Grassland/Herbaceous → Grassland
    81: 10,  # Pasture/Hay → Pasture
    82: 11,  # Cultivated Crops → Croplands
    90: 9,   # Woody Wetlands → Emergent Herbaceous Wetlands
    95: 8    # Emergent Herbaceous Wetlands → Woody Wetlands
}

In [16]:
# open the nlcd file using rioxarray
nlcd = rxr.open_rasterio(land_path / land_name)
# 
# Apply mapping using xarray where method
modis_da = nlcd.copy()
for nlcd_value, modis_value in nlcd_to_modis.items():
    modis_da = modis_da.where(nlcd != nlcd_value, modis_value)

In [20]:
# Save the modified raster
modis_da.rio.to_raster(land_path / 'landclass_usgs.tif')